In [ ]:
import numpy as np
import os
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Input, Conv2DTranspose, Cropping2D
import keras.utils.all_utils as kr_utils
import keras.regularizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa

In [ ]:
'''
imgdir = '../input/img-data/images/images'
maskdir = '../input/img-data/masks/masks'
imgs = []
masks = []

for filename in os.listdir(imgdir):
    imgpath = os.path.join(imgdir, filename)
    maskpath = os.path.join(maskdir, filename)
    imginst = cv2.imread(imgpath)
    maskinst = cv2.imread(maskpath)
    
    imginst = cv2.cvtColor(imginst, cv2.COLOR_BGR2RGB)
    imgind = np.nonzero(maskinst > 0)
    maskinst[imgind[0],imgind[1],:] = 255
    
    imgs.append(cv2.imread(imgpath))
    masks.append(cv2.imread(maskpath))

masks = np.array(masks)
masks = np.expand_dims(masks[:,:,:,0], axis = 3)
imgs = np.array(imgs)
'''

In [ ]:
np.random.seed = 24
drop_ratio = 8/100
max_conv_maps = 256

def encoders(mod_input, conv_maps, filter_sz, padding_type):
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(mod_input)
    ei = Dropout(drop_ratio)(ei)
    ei = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(ei)
    return ei

def decoders(enc_input, up_input, up_maps, up_sz, up_stride, padding_type, conv_maps, filter_sz):
    up_do = Conv2DTranspose(up_maps, up_sz, strides=up_stride, padding = padding_type)(up_input)
    concat = Concatenate()([enc_input,up_do])
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(concat)
    up_do = Conv2D(conv_maps,filter_sz,padding = padding_type, activation='relu')(up_do)
    return up_do

cnn_input = Input(shape=(128, 128, 3))
eo1 = encoders(cnn_input, max_conv_maps / (2 ** 4), (3,3), 'same')
ei2 = MaxPooling2D((2,2))(eo1)

eo2 = encoders(ei2, max_conv_maps / (2 ** 3), (3,3), 'same')
ei3 = MaxPooling2D((2,2))(eo2)

eo3 = encoders(ei3, max_conv_maps / (2 ** 2), (3,3), 'same')
ei4 = MaxPooling2D((2,2))(eo3)

eo4 = encoders(ei4, max_conv_maps / (2 ** 1), (3,3), 'same')
ei5 = MaxPooling2D((2,2))(eo4)

peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(ei5)
peak = Dropout(drop_ratio)(peak)
peak = Conv2D(max_conv_maps / (2 ** 0),(3,3),padding = 'same', activation='relu')(peak)

do4 = decoders(eo4, peak, max_conv_maps / (2 ** 1), (2,2), (2,2), 'same', 128, (3,3))
do3 = decoders(eo3, do4, max_conv_maps / (2 ** 2), (2,2), (2,2), 'same', 64, (3,3))
do2 = decoders(eo2, do3, max_conv_maps / (2 ** 3), (2,2), (2,2), 'same', 32, (3,3))
do1 = decoders(eo1, do2, max_conv_maps / (2 ** 4), (2,2), (2,2), 'same', 16, (3,3))

fin_out = Conv2D(1, (1,1), padding = 'same', activation='sigmoid')(do1)

model = Model(inputs = cnn_input, outputs = fin_out)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(imgs,masks, epochs=10, verbose=1)